In [2]:
import numpy as np
import pandas as pd
import xarray as xr
import os

In [3]:
file_path="../data/RIDE_DATA_OCT_2016-Bus2-wo-CAN.csv"
df = pd.DataFrame.from_csv(file_path, index_col=None)
df = df[df["Direction"]==0]

In [4]:
df['Ride_start_date'] = df['Ride_start_date'].values.astype('datetime64[D]')
df['Boarding_stop_stn'] = df['Boarding_stop_stn'].astype("str")
df['Alighting_stop_stn'] = df['Alighting_stop_stn'].astype('str')
dates = pd.Series.unique(df["Ride_start_date"])
dates = dates.astype('datetime64[D]')
dates = np.sort(dates)
print(dates)
stops_raw = ["10589","10017","05013","05023","04222","04142","01012","01113","01121","01211","01311","80011","80031","80051","80071","80091","81011","81031","81051","82011","82032","82061","83011","83031","83062","83081","84011","84021","84031","84041","84051","84061","85091","85051","85061","85081","85071","96071","96081","96091","96041","96051","96061","97011","97031","97041","97051","97201","97061","97071","97081","97091","98061","98071","99011","99021","99031","99041","99139","99009"]
stops =[]
for s in stops_raw:
    char_list = list(s)
    while char_list[0] == '0':
        char_list.pop(0)
        
    code_modified = ''.join(char_list)
    stops.append(code_modified)

print(stops)

['2016-10-01' '2016-10-02' '2016-10-03' '2016-10-04' '2016-10-05'
 '2016-10-06' '2016-10-07' '2016-10-08' '2016-10-09' '2016-10-10'
 '2016-10-11' '2016-10-12' '2016-10-13' '2016-10-14' '2016-10-15'
 '2016-10-16' '2016-10-17' '2016-10-18' '2016-10-19' '2016-10-20'
 '2016-10-21' '2016-10-22' '2016-10-23' '2016-10-24' '2016-10-25'
 '2016-10-26' '2016-10-27' '2016-10-28' '2016-10-29' '2016-10-30'
 '2016-10-31']
['10589', '10017', '5013', '5023', '4222', '4142', '1012', '1113', '1121', '1211', '1311', '80011', '80031', '80051', '80071', '80091', '81011', '81031', '81051', '82011', '82032', '82061', '83011', '83031', '83062', '83081', '84011', '84021', '84031', '84041', '84051', '84061', '85091', '85051', '85061', '85081', '85071', '96071', '96081', '96091', '96041', '96051', '96061', '97011', '97031', '97041', '97051', '97201', '97061', '97071', '97081', '97091', '98061', '98071', '99011', '99021', '99031', '99041', '99139', '99009']


In [5]:
# runs = df[(df['Boarding_stop_stn']=="1113") & (df['Ride_start_date']==dates[0])]
runs = df[df['Ride_start_date']==dates[0]]
runs = runs[['Bus_Reg_Num', 'Bus_Trip_Num']].drop_duplicates()
num_runs = runs.shape[0]
print(num_runs)

116


In [6]:
stop_selected = ""
date = dates[0]
max_runs = 0
for s in stops:
    stop_runs = df[((df['Boarding_stop_stn']==s) | (df['Alighting_stop_stn']==s)) & (df['Ride_start_date']==date)]
    stop_runs = stop_runs[['Bus_Reg_Num', 'Bus_Trip_Num']].drop_duplicates()
    num_stop_runs = stop_runs.shape[0]
    if num_stop_runs > max_runs:
        max_runs = num_stop_runs
        stop_selected = s
    
print(stop_selected)
print(max_runs)


84031
110


In [7]:
stop_max_runs = df[((df['Boarding_stop_stn']==stop_selected) | (df['Alighting_stop_stn']==stop_selected)) & (df['Ride_start_date']==date)]
stop_max_runs = stop_max_runs[['Bus_Reg_Num', 'Bus_Trip_Num']].drop_duplicates()
stop_runs_index = stop_max_runs.index

In [8]:
all_runs = df.loc[stop_runs_index]

In [9]:
all_runs['Time'] = np.where(all_runs['Boarding_stop_stn']==stop_selected, all_runs['Ride_start_time'], all_runs['Ride_end_time'])

In [10]:
all_runs.sort_values(by='Time', inplace=True)

In [ ]:
runs_array = all_runs[['Time', 'Bus_Reg_Num', 'Bus_Trip_Num']].to_xarray()

In [14]:
df_day = df[df['Ride_start_date']==date]
records = pd.merge(df_day, all_runs, how="inner", on=['Bus_Reg_Num', 'Bus_Trip_Num'])

In [ ]:
records.groupby(['Bus_Reg_Num', 'Bus_Trip_Num', 'Boarding_stop_stn_x']).size()

Bus_Reg_Num  Bus_Trip_Num  Boarding_stop_stn_x
2            1             10017                   5
                           1012                    1
                           1113                    7
                           1121                    4
                           1211                    3
                           1311                   10
                           4142                    2
                           4222                    3
                           5013                   26
                           80011                   3
                           80031                   1
                           80051                   2
                           80071                   1
                           80091                   6
                           81011                   2
                           81031                   8
                           81051                   4
                           82011                   2